# Poetry Generation with LSTM & GRU (Keras)
This notebook explores how to build and train a Recurrent Neural Network (RNN) using LSTM and GRU layers to generate poetry. 

Models are trained on a custom dataset of poems and learns to predict and generate poetic lines based on input text.

- Framework: TensorFlow / Keras  
- Architecture: Word-level LSTM / GRU 
- Dataset: Text file (`poem.txt`) https://www.kaggle.com/datasets/harshalgadhe/poem-generation
- Goal: Generate poetry using AI with adjustable creativity (temperature sampling)


In [47]:
# import necessary libraries
import pandas as pd
import numpy as np
import random

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GRU, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

## Load data

In [48]:
# Step 1: Read the TXT file
with open("poem.txt", "r", encoding="utf-8") as file:
    text = file.read().lower()

In [52]:
print(text[:200])

stay, i said
to the cut flowers.
they bowed
their heads lower.
stay, i said to the spider,
who fled.
stay, leaf.
it reddened,
embarrassed for me and itself.
stay, i said to my body.
it sat as a dog do


## Word Tokenization

In [53]:
# create instance of tokenizer
tokenizer = Tokenizer()
# fit tokenizer to current text
tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1
print(f"Total unique words: {total_words}")

# # Generate input sequences using n-gram approach
input_sequences = []
for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

print(f"Total training sequences: {len(input_sequences)}")

Total unique words: 3808
Total training sequences: 16311


## Pad Sequences and Prepare Features 

In [55]:
# pad sequences
max_seq_len = max(len(seq) for seq in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')

# x contains all words except  the last word (to be generated)
X = input_sequences[:, :-1]
y = input_sequences[:, -1] # word to be generated for each line
y = np.eye(total_words)[y]  # one-hot encode the labels

## Build and Train First GRU Layer

In [57]:
# define model architecture
gru_model = Sequential([
    Embedding(total_words, 100, input_length=max_seq_len - 1),
    GRU(128, return_sequences=True),
    Dropout(0.2),
    GRU(128),
    Dropout(0.2),
    Dense(total_words, activation='softmax')
])

gru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
gru_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 15, 100)           380800    
                                                                 
 gru_12 (GRU)                (None, 15, 128)           88320     
                                                                 
 dropout_20 (Dropout)        (None, 15, 128)           0         
                                                                 
 gru_13 (GRU)                (None, 128)               99072     
                                                                 
 dropout_21 (Dropout)        (None, 128)               0         
                                                                 
 dense_10 (Dense)            (None, 3808)              491232    
                                                                 
Total params: 1,059,424
Trainable params: 1,059,424
N

In [58]:
# fit model
gru_model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
510/510 [==============================] - 12s 13ms/step - loss: 7.0930 - accuracy: 0.0611
Epoch 2/50
510/510 [==============================] - 6s 12ms/step - loss: 6.5906 - accuracy: 0.0661
Epoch 3/50
510/510 [==============================] - 6s 11ms/step - loss: 6.3486 - accuracy: 0.0764
Epoch 4/50
510/510 [==============================] - 6s 12ms/step - loss: 6.1170 - accuracy: 0.0891
Epoch 5/50
510/510 [==============================] - 6s 11ms/step - loss: 5.8652 - accuracy: 0.0994
Epoch 6/50
510/510 [==============================] - 6s 12ms/step - loss: 5.5945 - accuracy: 0.1116
Epoch 7/50
510/510 [==============================] - 6s 11ms/step - loss: 5.3271 - accuracy: 0.1208
Epoch 8/50
510/510 [==============================] - 6s 11ms/step - loss: 5.0481 - accuracy: 0.1351
Epoch 9/50
510/510 [==============================] - 6s 11ms/step - loss: 4.7744 - accuracy: 0.1498
Epoch 10/50
510/510 [==============================] - 6s 11ms/step - loss: 4.5107 - accur

## Model Testing

In [59]:

# defien funcitons to generate text using trained model
def sample_with_temperature(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-10) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)

def generate_poem(seed_text, model, next_words=30, temperature=1.0):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len - 1, padding='pre')
        predicted = model.predict(token_list, verbose=0)[0]
        predicted_index = sample_with_temperature(predicted, temperature)
        output_word = tokenizer.index_word.get(predicted_index, "")
        seed_text += " " + output_word
    return seed_text


In [61]:
print(generate_poem("i said to myself", model=gru_model, next_words=20, temperature=0.4))

i said to myself and has only with tease me out of thought with a summers wall to the wearin o the green and


##### model doesn't seem to be doing well, based on accuracy, it looks to underfit

## LSTM Model

In [62]:
# lstm model with same architecture as gru model, and early stopping applied
lstm_model = Sequential([
    Embedding(total_words, 100, input_length=max_seq_len - 1),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(128),
    Dropout(0.3),
    Dense(total_words, activation='softmax')
])

early_stopping_monitor = EarlyStopping(monitor='loss',patience = 5)

lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.summary()

lstm_model.fit(X, y, epochs=50, verbose=1, callbacks=[early_stopping_monitor])

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 15, 100)           380800    
                                                                 
 lstm_8 (LSTM)               (None, 15, 128)           117248    
                                                                 
 dropout_22 (Dropout)        (None, 15, 128)           0         
                                                                 
 lstm_9 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_23 (Dropout)        (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 3808)              491232    
                                                                 
Total params: 1,120,864
Trainable params: 1,120,864
N

#### lstm_model looks to underfit even more than the gru model

In [64]:
print(generate_poem("i said to myself", model=lstm_model, next_words=20, temperature=0.4))

i said to myself to van diemans land is a jewel when so deep as gone to darlin troubled gone wandered gone gone by


## Bi-directional GRU Model

In [65]:
# define model architecture
gru_model2 = Sequential([
    Embedding(total_words, 100, input_length=max_seq_len - 1),
    Bidirectional(GRU(128, return_sequences=True)),
    Dropout(0.4),
    Bidirectional(GRU(128)),
    Dropout(0.4),
    Dense(total_words, activation='softmax')
])

early_stopping_monitor = EarlyStopping(monitor='loss',patience = 5)

gru_model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
gru_model2.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 15, 100)           380800    
                                                                 
 bidirectional_5 (Bidirectio  (None, 15, 256)          176640    
 nal)                                                            
                                                                 
 dropout_24 (Dropout)        (None, 15, 256)           0         
                                                                 
 bidirectional_6 (Bidirectio  (None, 256)              296448    
 nal)                                                            
                                                                 
 dropout_25 (Dropout)        (None, 256)               0         
                                                                 
 dense_12 (Dense)            (None, 3808)            

In [66]:
# fit model
gru_model2.fit(X, y, epochs=50, verbose=1, callbacks=[early_stopping_monitor])

Epoch 1/50
510/510 [==============================] - 9s 11ms/step - loss: 6.8761 - accuracy: 0.0641
Epoch 2/50
510/510 [==============================] - 9s 17ms/step - loss: 6.4228 - accuracy: 0.0777
Epoch 3/50
510/510 [==============================] - 5s 10ms/step - loss: 6.1512 - accuracy: 0.0927
Epoch 4/50
510/510 [==============================] - 5s 10ms/step - loss: 5.8980 - accuracy: 0.1066
Epoch 5/50
510/510 [==============================] - 5s 10ms/step - loss: 5.6253 - accuracy: 0.1192
Epoch 6/50
510/510 [==============================] - 6s 11ms/step - loss: 5.3588 - accuracy: 0.1343
Epoch 7/50
510/510 [==============================] - 5s 11ms/step - loss: 5.0597 - accuracy: 0.1504
Epoch 8/50
510/510 [==============================] - 6s 11ms/step - loss: 4.7640 - accuracy: 0.1688
Epoch 9/50
510/510 [==============================] - 5s 11ms/step - loss: 4.4806 - accuracy: 0.1925
Epoch 10/50
510/510 [==============================] - 6s 11ms/step - loss: 4.2047 - accura

In [69]:
print(generate_poem("i said to myself", model=gru_model2, next_words=20, temperature=0.7))

i said to myself i see another free goodbye to dear old skibbereen and killarney come home they took a tired boy i love


#### looks to perform better than previous gru model